In [2]:
!pip install yfinance pandas statsmodels matplotlib

In [3]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.ar_model import AutoReg
from sklearn.preprocessing import MinMaxScaler
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

In [4]:
nifty50 = [
    'RELIANCE.NS', 'TCS.NS', 'HDFCBANK.NS', 'INFY.NS', 'HINDUNILVR.NS',
    'ICICIBANK.NS', 'SBIN.NS', 'BHARTIARTL.NS', 'ITC.NS', 'ASIANPAINT.NS',
    'MARUTI.NS', 'HCLTECH.NS', 'KOTAKBANK.NS', 'LT.NS', 'AXISBANK.NS',
    'TITAN.NS', 'SUNPHARMA.NS', 'ULTRACEMCO.NS', 'NESTLEIND.NS', 'WIPRO.NS',
    'JSWSTEEL.NS', 'M&M.NS', 'NTPC.NS', 'POWERGRID.NS', 'TATAMOTORS.NS',
    'TECHM.NS', 'HDFCLIFE.NS', 'SBILIFE.NS', 'BAJFINANCE.NS', 'COALINDIA.NS',
    'HINDALCO.NS', 'GRASIM.NS', 'BRITANNIA.NS', 'SHREECEM.NS', 'CIPLA.NS',
    'DIVISLAB.NS', 'DRREDDY.NS', 'EICHERMOT.NS', 'HEROMOTOCO.NS', 'BAJAJFINSV.NS',
    'INDUSINDBK.NS', 'TATASTEEL.NS', 'ADANIENT.NS', 'APOLLOHOSP.NS', 'BPCL.NS',
    'ONGC.NS', 'TATACONSUM.NS', 'BAJAJ-AUTO.NS', 'ADANIPORTS.NS', 'UPL.NS'
]

data = yf.download(
    tickers = nifty50,
    start = "2022-01-01",
    end = "2024-09-01",
    interval = "1d",
    group_by = 'ticker'
)


YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  50 of 50 completed


In [5]:
def clean_data(ticker):
    df = data[ticker].copy()
    df['Ticker'] = ticker
    df = df.rename(columns={'Close': 'close'})
    return df[['close']]


full_df = pd.concat([clean_data(ticker) for ticker in nifty50], axis=1)
full_df.columns = nifty50
full_df = full_df.ffill().dropna()
print(full_df.head())



            RELIANCE.NS       TCS.NS  HDFCBANK.NS      INFY.NS  HINDUNILVR.NS  \
Date                                                                            
2022-01-03  1109.376343  3552.990967  1465.145386  1755.068359    2258.326172   
2022-01-04  1134.412720  3615.344238  1473.726196  1755.715820    2296.533936   
2022-01-05  1139.719971  3593.194824  1508.724121  1705.331787    2310.449219   
2022-01-06  1115.214355  3543.405518  1484.524414  1680.509644    2287.113037   
2022-01-07  1124.213623  3586.261963  1494.937012  1677.273926    2310.783936   

            ICICIBANK.NS     SBIN.NS  BHARTIARTL.NS      ITC.NS  \
Date                                                              
2022-01-03    741.368225  438.425690     681.297180  191.641876   
2022-01-04    749.269531  450.252441     687.358276  192.647736   
2022-01-05    764.005859  458.540405     689.871338  192.910156   
2022-01-06    761.097351  457.888550     700.120911  191.291992   
2022-01-07    769.047119  457.

Rolling Volatility

In [6]:
returns = full_df.pct_change()
rolling_vol = returns.rolling(window=20).std()

1. Rolling Mean Forecast

In [13]:
rolling_forecast_rm = rolling_vol.rolling(window=10).mean()

2. EWMA Forecast

In [8]:
lambda_ = 0.94
ewma_vol = (returns ** 2).ewm(alpha=1 - lambda_).mean() ** 0.5

3. AR(1) Model

In [17]:
ar_forecast_series = pd.DataFrame(index=rolling_vol.index, columns=rolling_vol.columns)

for col in rolling_vol.columns:
    series = rolling_vol[col].dropna()
    if len(series) < 10:
        continue  # Skip if not enough data

    for t in range(21, len(series)):
        train_series = series.iloc[:t]

        try:
            model = AutoReg(train_series, lags=1, old_names=False)
            model_fit = model.fit()
            pred = model_fit.predict(start=t, end=t)
            ar_forecast_series.at[series.index[t], col] = pred.values[0]
        except:
            continue
ar_forecast=ar_forecast_series
ar_forecast

,RELIANCE.NS,TCS.NS,HDFCBANK.NS,INFY.NS,HINDUNILVR.NS,ICICIBANK.NS,SBIN.NS,BHARTIARTL.NS,ITC.NS,ASIANPAINT.NS,...,INDUSINDBK.NS,TATASTEEL.NS,ADANIENT.NS,APOLLOHOSP.NS,BPCL.NS,ONGC.NS,TATACONSUM.NS,BAJAJ-AUTO.NS,ADANIPORTS.NS,UPL.NS
Date,,,,,,,,,,,,,,,,,,,,,
2022-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-01-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-01-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-08-26,0.011216,0.014509,0.012838,0.016449,0.006685,0.009816,0.015113,0.015294,0.0106,0.012269,...,0.012211,0.021718,0.019123,0.015176,0.018426,0.027787,0.01176,0.015378,0.020819,0.018874
2024-08-27,0.011095,0.014275,0.012985,0.01518,0.006683,0.00983,0.014515,0.01171,0.009117,0.011922,...,0.01151,0.020461,0.017374,0.012358,0.018511,0.028423,0.012526,0.014867,0.01896,0.018084
2024-08-28,0.011105,0.014271,0.012835,0.015395,0.008192,0.010048,0.014217,0.010569,0.008974,0.011972,...,0.011422,0.020462,0.01736,0.012354,0.017882,0.028419,0.012465,0.014866,0.018915,0.017884


In [19]:
errors=[]
rmse_rolling=np.sqrt(((rolling_vol-rolling_forecast_rm)**2).mean())
errors.append(rmse_rolling.mean())

rmse_ewma=np.sqrt(((rolling_vol-ewma_vol)**2).mean())
errors.append(rmse_ewma.mean())

diff=rolling_vol-ar_forecast
squared=diff**2
mean=squared.mean().mean()
Ar_rmse=np.sqrt(mean)
errors.append(Ar_rmse)
errors

[np.float64(0.0027695173074730062),
 np.float64(0.0025034680983733676),
 np.float64(0.002128390072431889)]

In [20]:
errors = []

# RMSE for Rolling Mean forecast
rmse_rolling = np.sqrt(((rolling_vol - rolling_forecast_rm) ** 2).mean())
errors.append(rmse_rolling.mean())  # average RMSE across all stocks

# RMSE for EWMA forecast
rmse_ewma = np.sqrt(((rolling_vol - ewma_vol) ** 2).mean())
errors.append(rmse_ewma.mean())  # average RMSE

# RMSE for AR(1) forecast
diff = rolling_vol - ar_forecast
squared_diff = diff ** 2
mean_squared_error = squared_diff.mean().mean()  # mean over time and tickers
rmse_ar = np.sqrt(mean_squared_error)
errors.append(rmse_ar)

# Identify best method
methods = ['Rolling Mean', 'EWMA', 'AR(1)']
min_rmse = min(errors)
best_method = methods[errors.index(min_rmse)]

print("RMSEs:")
for m, e in zip(methods, errors):
    print(f"{m}: {e:.6f}")

print(f"\nBest method based on RMSE: {best_method} (RMSE = {min_rmse:.6f})")


RMSEs:
Rolling Mean: 0.002770
EWMA: 0.002503
AR(1): 0.002128

Best method based on RMSE: AR(1) (RMSE = 0.002128)


In [35]:
normalized_ar_forecast = ar_forecast_series.copy()

for col in normalized_ar_forecast.columns:
    series = normalized_ar_forecast[col].astype(float)
    min_val = series.min()
    max_val = series.max()

    if pd.notna(min_val) and pd.notna(max_val) and max_val != min_val:
        normalized_ar_forecast[col] = (series - min_val) / (max_val - min_val)
    else:

        normalized_ar_forecast[col] = np.nan


In [31]:
risk_score = 0.5
latest_normalized_vol = normalized_ar_forecast.iloc[-1]

# Compute distance from investor's score
distance = (latest_normalized_vol - risk_score).abs()

# Drop NaNs (in case some volatilities weren't available)
distance = distance.dropna()

# Get 3 stocks with minimum distance
top_3_stocks = distance.nsmallest(3)

top_3_stocks


,2024-08-30
HINDALCO.NS,0.000402
TATAMOTORS.NS,0.001201
EICHERMOT.NS,0.005825


In [38]:
initial_investment = 1000000
weights = 0.33
top_3_stocks['investment'] = (weights * initial_investment)
top_3_stocks['normalized_ar_forecast'] = normalized_ar_forecast.iloc[-1]


,2024-08-30
HINDALCO.NS,0.000402
TATAMOTORS.NS,0.001201
EICHERMOT.NS,0.005825
investment,330000.0
normalized_ar_forecast,RELIANCE.NS 0.27 TCS.NS 0.46 HD...


In [41]:
hindal=yf.download("HINDALCO.NS", start='2022-01-01', end='2025-01-01')
tata=yf.download("TATAMOTORS.NS", start='2022-01-01', end='2025-01-01')
eicher=yf.download("EICHERMOT.NS", start='2022-01-01', end='2025-01-01')

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [42]:
import plotly.graph_objects as go
def atr(df):
    df.columns = df.columns.get_level_values(0)
    high_low = df['High'] - df['Low']
    high_close_prev = abs(df['High'] - df['Close'].shift(1))
    low_close_prev = abs(df['Low'] - df['Close'].shift(1))
    df['ema'] = df['Close'].ewm(span=14, adjust=False).mean()

    df['True_Range'] = pd.concat([high_low, high_close_prev, low_close_prev], axis=1).max(axis=1)
    df['ATR'] = df['True_Range'].ewm(span=14, adjust=False).mean()

    k = 1.2

    df['up_band'] = df['ema'] + k * df['ATR']
    df['down_band'] = df['ema'] - k * df['ATR']

    df['Signal'] = 0
    df.loc[df['Close'] > df['up_band'], 'Signal'] = 1
    df.loc[df['Close'] < df['down_band'], 'Signal'] = -1

    last_signal = 0
    for i in range(len(df)):
        current = df.loc[df.index[i], 'Signal']
        if current != 0:
            if current == last_signal:
                df.loc[df.index[i], 'Signal'] = 0
            else:
                last_signal = current
    i=0
    while df['Signal'].iloc[i]==0:
      i=i+1
      if df['Signal'].iloc[i] == -1 : df['Signal'].iloc[i] = 0

    buy_df = df[df['Signal'] == 1]
    sell_df = df[df['Signal'] == -1]
    fig=go.Figure()
    fig.add_trace(go.Scatter(x=buy_df.index, y=buy_df['Close'], mode='markers', name='Buy', marker=dict(color='green', symbol='triangle-up', size=12)))
    fig.add_trace(go.Scatter(x=sell_df.index, y=sell_df['Close'], mode='markers', name='Sell', marker=dict(color='red', symbol='triangle-down', size=12)))
    fig.add_trace(go.Scatter(x=df.index, y=df['Close'], name='Close Price', line=dict(color='blue')))
    fig.add_trace(go.Scatter(x=df.index, y=df['up_band'], name='UP_band', line=dict(color='orange'),opacity=0.5))
    fig.add_trace(go.Scatter(x=df.index, y=df['down_band'], name='DOWN_band', line=dict(color='black'),opacity=0.5))
    fig.show()
    return df

In [43]:
tata_df=atr(tata)
tata_df=tata_df.reset_index()

In [44]:
eicher_df=atr(eicher)
eicher_df=eicher_df.reset_index()

<ipython-input-42-1535520527>:32: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [46]:
hindal_df=atr(hindal)
hindal_df=hindal_df.reset_index()

In [47]:
def backtesting(df,signal,initial_capital):
    trade_idx=[]
    opening_trade_idx=[]
    closing_trade_idx=[]
    entry_date=[]
    exit_date=[]
    pos=[]
    type_of_trade=[]

    prev_pos=0
    current_signal=0
    for i in range(len(df)):
        if df.loc[i,signal]==1 or df.loc[i,signal]==-1:
            trade_idx.append(i)

        current_signal=df.loc[i,signal]

        if current_signal==1:
            if prev_pos==0:
                pos.append(1)
                opening_trade_idx.append(i)
                type_of_trade.append("long")
                entry_date.append(df.loc[i,"Date"])
            elif prev_pos==1:
                pos.append(1)
            elif prev_pos==-1:
                pos.append(0)
                closing_trade_idx.append(i)
                exit_date.append(df.loc[i,"Date"])
        elif current_signal==-1:
            if prev_pos==0:
                pos.append(-1)
                opening_trade_idx.append(i)
                type_of_trade.append("short")
                entry_date.append(df.loc[i,"Date"])
            elif prev_pos==1:
                pos.append(0)
                closing_trade_idx.append(i)
                exit_date.append(df.loc[i,"Date"])
            elif prev_pos==-1:
                pos.append(-1)
        else:
            pos.append(prev_pos)

        prev_pos=pos[i]

    #squaring off all positions at the end
    if pos[len(df)-1]==1:
        pos[len(df)-1]=0
        df.loc[len(df)-1,signal]=-1
        closing_trade_idx.append(len(df)-1)
        exit_date.append(df.loc[len(df)-1,"Date"])
    elif pos[len(df)-1]==-1:
        pos[len(df)-1]=0
        df.loc[len(df)-1,signal]=1
        closing_trade_idx.append(len(df)-1)
        exit_date.append(df.loc[len(df)-1,"Date"])


#STOP LOSS FOR LONG TRADES
    for k in range(len(opening_trade_idx)):
        if type_of_trade[k]=="long":
            entry_row=opening_trade_idx[k]
            planned_out=closing_trade_idx[k]
            stop_loss=0.1
            entry_price=df.loc[entry_row,"Close"]
            sl=(1- stop_loss) * entry_price

            for j in range(entry_row,planned_out):
                if df.loc[j,"Close"]< sl:
                    closing_trade_idx[k]=j
                    df.loc[j,signal]=-1
                    df.loc[planned_out,signal]=0
                    break

#DYNAMIC EXIT FOR SHORT TRADES
    for k in range(len(opening_trade_idx)):
        if type_of_trade[k]=="short":
            entry_row=opening_trade_idx[k]
            planned_out=closing_trade_idx[k]
            dynamic_exit=0.1
            entry_price=df.loc[entry_row,"Close"]
            de=(1+ dynamic_exit) * entry_price

            for j in range(entry_row,planned_out):
                if df.loc[j,"Close"]> de:
                    closing_trade_idx[k]=j
                    df.loc[j,signal]=1
                    df.loc[planned_out,signal]=0
                    break


    no_of_shares=[]
    capital=initial_capital
    portfolio=[]
    shares=0

    for i in range(len(df)):
        price=df.loc[i,"Close"]
        #long trade
        if df.loc[i,signal]==1 and pos[i-1]==0 and capital>price :
            shares=int(capital/price)
            capital-=shares*price
        elif df.loc[i,signal]==-1 and pos[i-1]==1 and shares>0:
            capital+=shares*price
            shares=0
        #short trade
        elif df.loc[i,signal]==-1 and pos[i-1]==0 and capital>price:
            shares=-int(capital/price)
            capital-=shares*price
        elif df.loc[i,signal]==1 and pos[i-1]==-1 and shares<0:
            capital+=shares*price
            shares=0

        portfolio_value=shares*price+capital
        portfolio.append(portfolio_value)
        no_of_shares.append(shares)


    returns=[]
    for i in range(len(opening_trade_idx)):
        if type_of_trade[i]=="long":
            returns.append((df.loc[closing_trade_idx[i],"Close"]-df.loc[opening_trade_idx[i],"Close"])*100/df.loc[opening_trade_idx[i],"Close"])
        else:
            returns.append(-(df.loc[closing_trade_idx[i],"Close"]-df.loc[opening_trade_idx[i],"Close"])*100/df.loc[opening_trade_idx[i],"Close"])



    trade_wise_df=pd.DataFrame({"Entry Index": opening_trade_idx,"Exit Index":closing_trade_idx,"Entry Date":entry_date,"Exit Date":exit_date,"Type of Trade":type_of_trade})
    trade_wise_df["No of stock traded"]=[no_of_shares[i]-no_of_shares[i-1] for i in opening_trade_idx]
    trade_wise_df["Return for trade in %"]=returns
    trade_wise_df["Trade Duration"]=trade_wise_df["Exit Index"]-trade_wise_df["Entry Index"]
    #np.where(df_1["Type of trade"]=='long',(returns,-returns)

    total_daily=pd.DataFrame({"Portfolio Value":portfolio,"No. of Stock":no_of_shares})
    total_daily["Profit from initial Capital"]=(total_daily["Portfolio Value"]-initial_capital)*100/initial_capital
    total_daily["Daily Returns in %"]=(total_daily["Portfolio Value"].pct_change())*100
    return trade_wise_df,total_daily

In [68]:
tata_trade,tata_daily=backtesting(tata_df,"Signal",top_3_stocks["investment"])
tata_trade

,Entry Index,Exit Index,Entry Date,Exit Date,Type of Trade,No of stock traded,Return for trade in %,Trade Duration
0,10,15,2022-01-17,2022-01-24,long,637,-8.866664,5
1,63,83,2022-04-05,2022-05-06,long,666,-10.806673,20
2,99,109,2022-05-30,2022-06-13,long,615,-8.069615,10
3,127,168,2022-07-07,2022-09-07,long,581,3.481488,41
4,205,231,2022-11-01,2022-12-08,long,614,-0.984918,26
5,254,287,2023-01-10,2023-02-27,long,621,1.223058,33
6,312,580,2023-04-06,2024-05-13,long,593,119.979746,268
7,613,637,2024-07-01,2024-08-05,long,566,1.437052,24
8,654,661,2024-08-29,2024-09-09,long,513,-7.395368,7


In [69]:
eicher_trade,eicher_daily=backtesting(eicher_df,"Signal",top_3_stocks["investment"])
eicher_trade

,Entry Index,Exit Index,Entry Date,Exit Date,Type of Trade,No of stock traded,Return for trade in %,Trade Duration
0,63,82,2022-04-05,2022-05-05,long,133,-4.320906,19
1,89,212,2022-05-16,2022-11-11,long,124,35.337713,123
2,315,361,2023-04-12,2023-06-19,long,139,11.772982,46
3,456,494,2023-11-06,2024-01-02,long,137,10.867426,38
4,514,595,2024-02-01,2024-06-04,long,135,14.293704,81
5,603,636,2024-06-14,2024-08-02,long,123,-4.222811,33
6,649,679,2024-08-22,2024-10-04,long,117,-4.596082,30
7,699,707,2024-11-01,2024-11-13,long,111,-7.366358,8


In [70]:
hindal_trade,hindal_daily=backtesting(hindal_df,"Signal",top_3_stocks["investment"])
hindal_trade

,Entry Index,Exit Index,Entry Date,Exit Date,Type of Trade,No of stock traded,Return for trade in %,Trade Duration
0,4,60,2022-01-07,2022-03-31,long,683,15.400200,56
1,137,180,2022-07-21,2022-09-23,long,1034,6.393249,43
2,206,242,2022-11-02,2022-12-23,long,973,1.958229,36
3,248,272,2023-01-02,2023-02-06,long,858,-8.151110,24
4,314,335,2023-04-11,2023-05-12,long,919,-3.065499,21
5,379,447,2023-07-14,2023-10-23,long,833,2.848654,68
6,458,506,2023-11-08,2024-01-18,long,779,13.697380,48
7,519,522,2024-02-08,2024-02-13,long,719,-15.167138,3
8,550,626,2024-03-26,2024-07-19,long,656,18.678951,76
9,647,663,2024-08-20,2024-09-11,long,643,-3.744986,16


In [73]:
def metrics(price_df, trade_df, total_daily, capital):
    print("Initial Capital:", capital)

    # Gross return from all trades
    final_portfolio_value = total_daily["Portfolio Value"].iloc[-1]
    total_return_pct = (final_portfolio_value - capital) * 100 / capital
    print("Total Portfolio Return: {:.2f}%".format(total_return_pct))

    # Drawdown and Dip Calculations
    drawdowns = []
    dips = []
    returns_for_sharpe = []

    for i in range(len(trade_df)):
        entry_idx = trade_df["Entry Index"].iloc[i]
        exit_idx = trade_df["Exit Index"].iloc[i]

        initial_value = total_daily["Portfolio Value"].iloc[entry_idx]
        trade_range = total_daily["Portfolio Value"].iloc[entry_idx:exit_idx+1]
        min_value = trade_range.min()
        dip_pct = (min_value - initial_value) * 100 / initial_value
        dips.append(dip_pct)

        max_drawdown = 0
        peak = initial_value
        for val in trade_range:
            peak = max(peak, val)
            dd = (peak - val) / peak
            max_drawdown = max(max_drawdown, dd)
        drawdowns.append(100 * max_drawdown)

        # Return for Sharpe
        pnl = trade_df["Return for trade in %"].iloc[i] / 100
        returns_for_sharpe.append(pnl)

    # Benchmark: buy & hold
    number_of_stock = int(capital / price_df['Close'].iloc[0])
    benchmark_final = number_of_stock * price_df['Close'].iloc[-1]
    benchmark_return = benchmark_final - capital
    print("\nBenchmark Return (₹):", round(benchmark_return, 2))

    print("\nNumber of Closed Trades:", len(trade_df))
    print("Max Holding Time (days):", trade_df["Trade Duration"].max())
    print("Average Holding Time (days):", trade_df["Trade Duration"].mean())

    print("\nGross Profit (₹):", round(final_portfolio_value - capital, 2))
    print("Net Profit after ₹20 fee/trade (₹):", round(final_portfolio_value - capital - 20 * len(trade_df), 2))

    print("\nMax Drawdown (%):", round(np.max(drawdowns), 2))
    print("Average Drawdown (%):", round(np.mean(drawdowns), 2))
    print("Max Dip (%):", round(np.min(dips), 2))
    print("Average Dip (%):", round(np.mean(dips), 2))

    if np.std(returns_for_sharpe) != 0:
        sharpe = np.sqrt(252) * (np.mean(returns_for_sharpe) / np.std(returns_for_sharpe))
    else:
        sharpe = 0
    print("\nSharpe Ratio:", round(sharpe, 2))


In [77]:
trade_df, total_daily = backtesting(tata_df, 'Signal', 1000000)

metrics(tata_df, trade_df, total_daily, 1000000)

Initial Capital: 1000000
Total Portfolio Return: 60.14%

Benchmark Return (₹): 496300.93

Number of Closed Trades: 9
Max Holding Time (days): 268
Average Holding Time (days): 48.22222222222222

Gross Profit (₹): 601416.08
Net Profit after ₹20 fee/trade (₹): 601236.08

Max Drawdown (%): 12.51
Average Drawdown (%): 8.82
Max Dip (%): -10.81
Average Dip (%): -4.86

Sharpe Ratio: 4.05


In [78]:
trade_df, total_daily = backtesting(eicher_df, 'Signal', 1000000)

metrics(eicher_df, trade_df, total_daily, 1000000)

Initial Capital: 1000000
Total Portfolio Return: 55.22%

Benchmark Return (₹): 822678.16

Number of Closed Trades: 8
Max Holding Time (days): 123
Average Holding Time (days): 47.25

Gross Profit (₹): 552231.62
Net Profit after ₹20 fee/trade (₹): 552071.62

Max Drawdown (%): 9.18
Average Drawdown (%): 7.48
Max Dip (%): -7.36
Average Dip (%): -3.63

Sharpe Ratio: 7.54


In [79]:
trade_df, total_daily = backtesting(hindal_df, 'Signal', 1000000)

metrics(hindal_df, trade_df, total_daily, 1000000)

Initial Capital: 1000000
Total Portfolio Return: 21.53%

Benchmark Return (₹): 286833.23

Number of Closed Trades: 11
Max Holding Time (days): 76
Average Holding Time (days): 37.45454545454545

Gross Profit (₹): 215280.73
Net Profit after ₹20 fee/trade (₹): 215060.73

Max Drawdown (%): 15.16
Average Drawdown (%): 10.08
Max Dip (%): -15.16
Average Dip (%): -4.02

Sharpe Ratio: 3.61
